In [25]:
#general imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.model_selection import train_test_split

from autofeat import AutoFeatRegressor
import lightgbm as lgb
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import KFold

In [26]:
#Read dataset and make ydata-profiling dashboard

#from ydata_profiling import ProfileReport

df = pd.read_csv("../data/datasetRuntimeData.csv")
#Drop index col
df = df.drop(["Unnamed: 0"], axis=1)
#Remove special symbole
df['AutoML_adapter'] = df['AutoML_adapter'].str.replace(':', '')
#Apply one hot encoding
df = pd.get_dummies(df, columns=['AutoML_adapter'], prefix='', prefix_sep='')


#profile = ProfileReport(df, title="Profiling Report")
#profile.to_notebook_iframe()

In [27]:
df

,dataset_size_in_mb,dataset_rows,dataset_cols,missing_values,duplicated_rows,duplicated_cols,outliers,runtime_limit,autogluon,evalml,flaml,gama,h2o_automl,lama,pycaret,tpot
0,18.085659,539383,8,0,154622,0,99,40,False,False,False,False,True,False,False,False
1,18.085659,539383,8,0,154622,0,99,5,False,True,False,False,False,False,False,False
2,18.085659,539383,8,0,154622,0,99,640,False,False,False,True,False,False,False,False
3,18.085659,539383,8,0,154622,0,99,10,True,False,False,False,False,False,False,False
4,18.085659,539383,8,0,154622,0,99,20,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0.197092,683,36,0,41,0,0,20,False,False,False,False,True,False,False,False
316,0.197092,683,36,0,41,0,0,5,True,False,False,False,False,False,False,False
317,0.197092,683,36,0,41,0,0,5,False,False,False,False,False,False,False,True
318,0.197092,683,36,0,41,0,0,20,False,False,True,False,False,False,False,False


In [28]:
#AutoFeat for all automls


X = df.drop(["runtime_limit"], axis=1)
y = df["runtime_limit"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### get list of selected features ###
afreg = AutoFeatRegressor(verbose=1)

# X_train = afreg.fit_transform(X_train, y_train)
# X_test = afreg.transform(X_test)
X = afreg.fit_transform(X, y)

2025-02-05 19:24:33,282 INFO: [AutoFeat] The 2 step feature engineering process could generate up to 5565 features.
2025-02-05 19:24:33,283 INFO: [AutoFeat] With 320 data points this new feature matrix would use about 0.01 gb of space.
2025-02-05 19:24:33,285 INFO: [feateng] Step 1: transformation of original features


2025-02-05 19:24:35,185 INFO: [feateng] Generated 21 transformed features from 15 original features - done.
2025-02-05 19:24:35,187 INFO: [feateng] Step 2: first combination of features


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\numpy\core\_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
2025-02-05 19:24:35,808 INFO: [feateng] Generated 592 feature combinations from 630 original feature tuples - done.
2025-02-05 19:24:35,811 INFO: [feateng] Generated altogether 646 new features in 2 steps
2025-02-05 19:24:35,812 INFO: [feateng] Removing correlated features, as well as additions at the highest level
2025-02-05 19:24:35,825 INFO: [feateng] Generated a total of 357 additional features
2025-02-05 19:24:35,828 INFO: [featsel] Feature selection run 1/5


[featsel] Scaling data...done.        630 feature tuples combined


2025-02-05 19:24:37,976 INFO: [featsel] Feature selection run 2/5
2025-02-05 19:24:40,050 INFO: [featsel] Feature selection run 3/5
2025-02-05 19:24:42,131 INFO: [featsel] Feature selection run 4/5
2025-02-05 19:24:44,281 INFO: [featsel] Feature selection run 5/5
2025-02-05 19:24:46,315 INFO: [featsel] 20 features after 5 feature selection runs
c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\autofeat\featsel.py:270: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  if np.max(np.abs(correlations[c].ravel()[:i])) < 0.9:
2025-02-05 19:24:46,318 INFO: [featsel] 17 features after correlation filtering
2025-02-05 19:24:46,352 INFO: [featsel] 4 features after noise filtering
2025-02-05 19:24:46,353 INFO: [AutoFeat] Computing 2 new features.


2025-02-05 19:24:46,705 INFO: [AutoFeat]     2/    2 new features ...done.
2025-02-05 19:24:46,707 INFO: [AutoFeat] Final dataframe with 17 feature columns (2 new).
2025-02-05 19:24:46,707 INFO: [AutoFeat] Training final regression model.
2025-02-05 19:24:46,716 INFO: [AutoFeat] Trained model: largest coefficients:
2025-02-05 19:24:46,717 INFO: 61.25130949880728
2025-02-05 19:24:46,718 INFO: 135.148121 * gama
2025-02-05 19:24:46,718 INFO: 125.648121 * tpot
2025-02-05 19:24:46,718 INFO: 13.788905 * log(dataset_size_in_mb)
2025-02-05 19:24:46,719 INFO: 2.909237 * sqrt(duplicated_cols)*h2o_automl
2025-02-05 19:24:46,721 INFO: [AutoFeat] Final score: 0.1681


In [29]:
# #Create prediction plot
# def create_prediction_plot(y_test, predictions):
#     prediction_results = pd.DataFrame({
#         'runtime_limit_is': y_test,
#         'runtime_limit_predicted': predictions
#     })

#     best_case_x = [0, 5, 10, 20, 40, 80, 160, 320, 640]
#     best_case_y = [0, 5, 10, 20, 40, 80, 160, 320, 640]
#     plt.figure(figsize=(10, 6))
#     sns.scatterplot(
#         x='runtime_limit_predicted', 
#         y='runtime_limit_is', 
#         data=prediction_results, 
#         color='gray', marker='o'  # Using a distinct color palette
#     )

#     plt.plot(best_case_x, best_case_y)
#     plt.xscale('log', base=10)  # Logarithmic scale for x-axis
#     plt.yscale('log', base=10)  # Logarithmic scale for y-axis


#     # Find the limits in log space
#     x_min, x_max = 1, 100
#     y_min, y_max = 1, np.exp(6.6)

#     # Determine the limits to make them symmetrical in log space
#     log_min = min(np.log10(x_min), np.log10(y_min))
#     log_max = max(np.log10(x_max), np.log10(y_max))

#     # Apply the symmetrical limits
#     plt.xlim([10**log_min, 10**log_max])
#     plt.ylim([10**log_min, 10**log_max])




#     plt.xlabel('Optimal runtime predicted')
#     plt.ylabel('Optimal runtime measured')
#     #plt.legend(title='AutoML Solution', bbox_to_anchor=(1.05, 0.5), loc='center left')
#     #plt.title('Actual vs Predicted Runtime Limits')
#     plt.grid(True)
#     #plt.legend(title='Series')
#     plt.show()
    

In [30]:
def train_model(model, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        # Compute MAE
        mae = MAE(predictions, y_test)
        mae_scores.append(mae)
        
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model

In [31]:
# def train_model(model,):
#     model.fit(X_train, y_train)
#     predictions = pd.Series(model.predict(X_test))
    
#     # Compute MAE
#     mae = MAE(predictions, y_test)

#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))
#     return model

In [32]:
    
#Train custom ANN based on Keras
def train_keras(n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Define the model
        model = Sequential([
            Dense(len(X_train.columns), activation='relu', input_shape=(len(X_train.columns),)),  # Input layer (10 features)
            #Dense(12, activation='relu'),  # Hidden layer 1
            #Dense(6, activation='relu'),  # Hidden layer 2
            Dense(1, activation='linear')  # Output layer (for binary classification)
        ])

        # Define ModelCheckpoint callback
        checkpoint_callback = ModelCheckpoint(
            './keras_models/2_2_2_best_model.keras', 
            monitor='val_mae',  # Monitor validation loss
            save_best_only=True,  # Save only when validation loss improves
            mode='auto',  # We want to minimize validation loss
            #verbose=1
        )
        
        early_stopping_callback = EarlyStopping(
            monitor='val_mae',  # Monitor validation MAE
            patience=25,  # Stop training if the validation MAE doesn't improve for 10 epochs
            restore_best_weights=True  # Restore the model weights from the best epoch
        )

        # Compile the model
        model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
        model.fit(X_train, y_train, epochs=500, batch_size=16, verbose=0, callbacks=[checkpoint_callback, early_stopping_callback], validation_data=(X_test, y_test))


        model = tf.keras.models.load_model('./keras_models/2_2_2_best_model.keras')
        predictions = pd.Series(model.predict(X_test).flatten())

        # Compute MAE
        mae = MAE(predictions, y_test)
        mae_scores.append(mae)
    
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model


In [33]:
    
# #Train custom ANN based on Keras
# def train_keras():
#     import tensorflow as tf
#     from tensorflow import keras
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Dense, Convolution1D
#     from tensorflow.keras.callbacks import ModelCheckpoint

#     # Define the model
#     model = Sequential([
#         Dense(len(X_train.columns), activation='relu', input_shape=(len(X_train.columns),)),  # Input layer (10 features)
#         #Dense(12, activation='relu'),  # Hidden layer 1
#         #Dense(6, activation='relu'),  # Hidden layer 2
#         Dense(1, activation='linear')  # Output layer (for binary classification)
#     ])

#     # Define ModelCheckpoint callback
#     checkpoint_callback = ModelCheckpoint(
#         './keras_models/2_2_2_best_model.h5', 
#         monitor='val_mae',  # Monitor validation loss
#         save_best_only=True,  # Save only when validation loss improves
#         mode='auto',  # We want to minimize validation loss
#         #verbose=1
#     )

#     # Compile the model
#     model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
#     model.fit(X_train, y_train, epochs=500, batch_size=16, verbose=0, callbacks=[checkpoint_callback], validation_data=(X_test, y_test))


#     model = tf.keras.models.load_model('./keras_models/2_2_2_best_model.h5')
#     predictions = pd.Series(model.predict(X_test).flatten())

#     # Compute MAE
#     mae = MAE(predictions, y_test)

#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))

In [34]:

def train_lgbm_model(n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        train_data = lgb.Dataset(X_train, label=y_train)
        test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)



        # Define parameters for the LightGBM model
        params = {
            'objective': 'regression',  # Set the objective as regression
            'metric': 'mae',            # Use mean absolute error as the evaluation metric
            'verbose': 1                # Disable verbose output
        }

        # Train the LightGBM model
        num_round = 100
        model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

        # Make predictions on the test set
        predictions = model.predict(X_test, num_iteration=model.best_iteration)
        
        # Compute MAE
        mae = MAE(predictions, y_test)
        mae_scores.append(mae)

    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model

In [35]:

# def train_lgbm_model():
#     train_data = lgb.Dataset(X_train, label=y_train)
#     test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    

#     # Define parameters for the LightGBM model
#     params = {
#         'objective': 'regression',  # Set the objective as regression
#         'metric': 'mae',            # Use mean absolute error as the evaluation metric
#         'verbose': 1                # Disable verbose output
#     }

#     # Train the LightGBM model
#     num_round = 100
#     model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

#     # Make predictions on the test set
#     predictions = pd.Series(model.predict(X_test, num_iteration=model.best_iteration))
    
#     # Compute MAE
#     mae = MAE(predictions, y_test)

#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))
#     return model

In [36]:

models = { "Baseline": DummyRegressor(strategy="median"), 
          "LightGBM": None, 
          "Linear Regression": LinearRegression(), 
          "Decision Tree": DecisionTreeRegressor(random_state=42), 
          "Sklearn Neural Network": MLPRegressor(random_state=42), 
          "Keras Neural Network": None, 
          "Ridge": Ridge(), 
          "Lasso": Lasso(), 
          "Elastic": ElasticNet(), 
          "Random Forest": RandomForestRegressor(), 
          "Bayesian": BayesianRidge(), 
          "SVM": SVR()}


for model_name, model in models.items():
    if model_name == "LightGBM":
        models[model_name] = train_lgbm_model()
    elif model_name == "Keras Neural Network":
        models[model_name] = train_keras()
    else:
        models[model_name] = train_model(model)


###############################################
MAE Scores: [104.375, 105.15625, 155.15625, 121.5625, 106.40625, 93.4375, 51.71875, 85.3125, 75.46875, 109.84375]
<class 'sklearn.dummy.DummyRegressor'> Average MAE across 10 folds: 100.8438 ± 26.2209
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000132 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 234
[LightGBM] [Info] Number of data points in the train set: 288, number of used features: 16
[LightGBM] [Info] Start training from score 106.006944
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
###############################################
MAE Scores: [156.6584065966308, 107.99738436937332, 147.8694330751896, 344.5422780131921, 86.07381963729858, 234.21213340759277, 126.96505910158157, 186.22086569666862, 152.88631387799978, 143.1325967311859]
<class 'keras.src.models.sequential.Sequential'> Average MAE across 10 folds: 168.6558 ± 70.1824
###############################################
MAE Scores: [109.87435739773632, 112.66149215802176, 154.37838902996978, 164.3557917225828, 112.60591028332308, 103.33518354812068, 85.95105845501507, 102.86535667729405, 123.40802071726048, 121.85308111785086]
<class 'sklearn.linear_model._ridge.Ridge'> Average MAE across 10 folds: 119.1289 ± 22.5798
###############################################
MAE Scores: [109.07569255913205, 111.73548844597076, 155.78841039686566, 154.43473859133263, 111.45050020544588, 102.06372935653408, 86.89243692747829, 102.41228678946243, 122.2864180706021, 120.83854704109494]